In [6]:
import pandas as pd
from epilepsy_prediction import load_prediction_model,fuse_string,evaluate_model

In [7]:
def load_data(drug_name,file_name='/Users/yoavkt/Documents/epilepsy_data/{}_data.pkl'):
    data = pd.read_pickle(file_name.format(drug_name))
    return data['X'],data['y']

In [8]:
drug_name="carbamazepine"
clf_mk = load_prediction_model(drug_name)
drug_name="est_carba"
clf_bi = load_prediction_model(drug_name)
drug_name = 'carbamazepine'
X,y = load_data(drug_name)

In [4]:
eval = evaluate_model(clf_mk,clf_bi,X,y.astype(int))
rep = fuse_string(eval)

Results:

Metric auc1:
------------------------------------------------
org:
0.7411711952961847
mean:
0.7411077736443009
std:
0.012562845705715114
conf_interval:
95
conf_lower:
0.716107523484399
conf_upper:
0.7655803285889324

Metric auc2:
------------------------------------------------
org:
0.6307010834803918
mean:
0.6305634273351468
std:
0.01334549645748438
conf_interval:
95
conf_lower:
0.6042965468878836
conf_upper:
0.6566750406497799

Metric mcnemar_test:
------------------------------------------------
p_value:
0.03806366960413442
statistic:
71.0
n1:
99.0
n2:
71.0

Metric delong_test:
------------------------------------------------
p_value:
2.2683800040159284e-16
z:
8.206971173643009
auc1:
0.7411711952961847
auc2:
0.6307010834803918
cov11:
0.00015657571232559572
cov12:
7.731947510668091e-05
cov22:
0.00017924887655622966



In [5]:
print(rep)

auc1: 0.74 (upper: 0.77, lower: 0.72) 
 auc2: 0.63 (upper: 0.66, lower: 0.60) 
 mcnemar_test: p-value 0.038 
 delong_test: p-value 0.000 
 


In [18]:
drug_name="est_carba_new_out"
clf_binout = load_prediction_model(drug_name)
eval = evaluate_model(clf_mk,clf_binout,X,y.astype(int))
rep = fuse_string(eval)

Results:

Metric auc1:
------------------------------------------------
org:
0.7411711952961847
mean:
0.7411077736443009
std:
0.012562845705715114
conf_interval:
95
conf_lower:
0.716107523484399
conf_upper:
0.7655803285889324

Metric auc2:
------------------------------------------------
org:
0.6342011065987501
mean:
0.6342878831867886
std:
0.013836444200835131
conf_interval:
95
conf_lower:
0.6069219254510007
conf_upper:
0.6609032920542202

Metric mcnemar_test:
------------------------------------------------
p_value:
0.011751828037832798
statistic:
74.0
n1:
109.0
n2:
74.0

Metric delong_test:
------------------------------------------------
p_value:
7.136414911490942e-16
z:
8.068162287414328
auc1:
0.7411711952961847
auc2:
0.6342011065987501
cov11:
0.00015657571232559572
cov12:
8.703285448126036e-05
cov22:
0.00019327242561356883



In [19]:
print(rep)

auc1: 0.74 (upper: 0.77, lower: 0.72) 
 auc2: 0.63 (upper: 0.66, lower: 0.61) 
 mcnemar_test: p-value 0.012 
 delong_test: p-value 0.000 
 
